### Assignment 2 
#### SPARQL Queries

In [ ]:
from rdflib import Graph, Literal, URIRef
from rdflib.plugins.sparql import prepareQuery

In [ ]:
g = Graph()
g.parse('graph.ttl', format='turtle')

##### Provide categories, subcategories, and brand for a random item

In [ ]:
query_a = """
SELECT ?item ?category ?subcategory ?item_brand
WHERE {
  BIND (MNJP3NF-A/dp/B0BDJFLR1F/ref=sr_1_5?crid=377I992WX3F4U&keywords=apple+watch&qid=1696411168&sprefix=appl%2Caps%2C150&sr=8-5 AS ?item) .
  ?item rdf:type ?category .
  ?category rdfs:subClassOf <gp/browse.html?node=16241721031&ref_=nav_em_hb_beauty_0_2_16_2> .
  ?item brand ?item_brand .
}
LIMIT 1
"""

qres = g.query(query_a)
for row in qres:
    print(f"{row.fname} {row.lname}")
# OPTIONAL {
#     ?category ex:hasSubcategory ?subcategory .
#     ?subcategory rdfs:subClassOf* ex:Category .
#   }

##### Items from different subcategories with the same brand

In [ ]:
query_b = """
SELECT ?item1 ?item2 ?brand
WHERE {
  ?item1 rdf:type ex:Item .
  ?item2 rdf:type ex:Item .
  ?item1 ex:hasBrand ?brand .
  ?item2 ex:hasBrand ?brand .
  ?item1 ex:hasCategory ?category1 .
  ?item2 ex:hasCategory ?category2 .
  FILTER (?item1 != ?item2 && ?category1 != ?category2)
}
LIMIT 10
"""

qres = g.query(query_b)
for row in qres:
    print(f"{row.fname} {row.lname}")

##### Group products by brand and calculate average price or rating

In [ ]:
query_c = """
SELECT ?brand (AVG(?price) AS ?avgPrice) (AVG(?rating) AS ?avgRating)
WHERE {
  ?item rdf:type ex:Item .
  ?item ex:hasBrand ?brand .
  OPTIONAL { ?item ex:hasPrice ?price . }
  OPTIONAL { ?item ex:hasRating ?rating . }
}
GROUP BY ?brand
"""

qres = g.query(query_c)
for row in qres:
    print(f"{row.fname} {row.lname}")

##### Sort products in one category by average brand price or rating

In [ ]:
query_d = """
SELECT ?item ?brand (AVG(?price) AS ?avgPrice) (AVG(?rating) AS ?avgRating)
WHERE {
  ?item rdf:type ex:Item .
  ?item ex:hasBrand ?brand .
  ?item ex:hasCategory ex:SelectedCategory .
  OPTIONAL { ?item ex:hasPrice ?price . }
  OPTIONAL { ?item ex:hasRating ?rating . }
}
GROUP BY ?item ?brand
ORDER BY DESC(?avgPrice) # or DESC(?avgRating)
LIMIT 10
"""

qres = g.query(query_d)
for row in qres:
    print(f"{row.fname} {row.lname}")

#### Plain English Question: Find items frequently bought together with "ItemX."

In [ ]:
query_h = """
SELECT ?item
WHERE {
  ?order rdf:type ns:Order ;
         ns:contains ?itemX, ?itemY ;
         ns:orderedBy ?customer .

  ?order2 rdf:type ns:Order ;
          ns:contains ?itemX, ?itemZ ;
          ns:orderedBy ?customer .
  FILTER (?itemY != ?itemZ)
}
GROUP BY ?item
ORDER BY COUNT(?customer) DESC
"""

qres = g.query(query_d)
for row in qres:
    print(f"{row.fname} {row.lname}")